In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

#os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')
os.chdir('C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020')

## Define Functions

In [ ]:
# change to directory of raw 765 quarterly files
folder = "C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020\\data\\raw\\stock_quarterly_data"
interim_data = "data\\interim\\interim_quarterly_roe.csv"

In [ ]:
#print computation time script
def timing(f):
    def wrap(*args):
        time1 = time()
        ret = f(*args)
        time2 = time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

## Import Data

In [ ]:
data = []
for file in Path(folder).glob('*.csv'):
        df = pd.read_csv(file)
        stock_name = os.path.basename(file)
        stock_name_noext = stock_name.replace("_quarterly_financial_data.csv", "")
        df['Symbol'] = stock_name_noext
        data.append(df)

In [ ]:
bigframe = pd.concat(data)
print('initial shape {}'.format(bigframe.shape))

## Begin Cleaning

### Drop duplicates

In [ ]:
bigframe = bigframe.drop_duplicates()
print(print('new shape: {}'.format(bigframe.shape)))

### Replace "None" with NaN

In [ ]:
bigframe.replace('None', np.nan, inplace=True)

### Clean Ticker Names

In [ ]:
bigframe['Symbol'] = bigframe['Symbol'].astype(str)

In [ ]:
# replace ABI_0 with ABI
bigframe['Symbol'].replace({'ABI_0':'ABI'}, inplace = True)

# replace CCE_1 with CCE
bigframe['Symbol'].replace({'CCE_1':'CCE'}, inplace = True)

# DG_1 with DG
bigframe['Symbol'].replace({'DG_1':'DG'}, inplace = True)

# replace GAS_0 with GAS_new
bigframe['Symbol'].replace({'GAS_0':'GAS_new'}, inplace = True)

# replace GAS with GAS_old
bigframe['Symbol'].replace({'GAS':'GAS_old'}, inplace = True)

# replace GCI_0 with TEGNA
bigframe['Symbol'].replace({'GCI_0':'TEGNA'}, inplace = True)

#replace GGP_0 with GGP
bigframe['Symbol'].replace({'GGP_0':'GGP'}, inplace = True)

#replace GM_0 with GM
bigframe['Symbol'].replace({'GM_0':'GM'}, inplace = True)

#replace LEH_0 with LEH
bigframe['Symbol'].replace({'LEH_0':'LEH'}, inplace = True)

#replace MNST_0 with MWW
bigframe['Symbol'].replace({'MNST_0':'MWW'}, inplace = True)

# replace Q_1 with IQV
bigframe['Symbol'].replace({'Q_1':'IQV'}, inplace = True)

# replace WM_0 with WM_old
bigframe['Symbol'].replace({'WM_0':'WM_old'}, inplace = True)

#replace PLD with PLD_old
bigframe['Symbol'].replace({'PLD':'PLD_old'}, inplace = True)

#replace PLD_0 with PLD_new
bigframe['Symbol'].replace({'PLD_0':'PLD_new'}, inplace = True)

### Transform column into float

In [ ]:
bigframe['ROE'] = bigframe['ROE'].astype(float)

### Quarter End

In [ ]:
bigframe['Quarter end'] = pd.to_datetime(bigframe['Quarter end'], errors='ignore')
bigframe.head()

In [ ]:
finalframe = bigframe[['Quarter end', 'ROE', 'Symbol']]
finalframe.head()

## Convert Dates to Quarters

In [ ]:
finalframe['Quarter end'] = pd.to_datetime(finalframe['Quarter end'], infer_datetime_format = True)
finalframe.dtypes

In [ ]:
# period filters
periods_2 = [(finalframe['Quarter end'] >='10/22/2019'), 
            (finalframe['Quarter end'] < '10/22/2019') & (finalframe['Quarter end'] >= '7/22/2019'),
            (finalframe['Quarter end'] < '7/22/2019') & (finalframe['Quarter end'] >= '4/25/2019'),
            (finalframe['Quarter end'] < '4/25/2019') & (finalframe['Quarter end'] >= '1/22/2019'),
            (finalframe['Quarter end'] < '1/22/2019') & (finalframe['Quarter end'] >= '10/22/2018'),
            (finalframe['Quarter end'] < '10/22/2018') & (finalframe['Quarter end'] >= '7/22/2018'),
            (finalframe['Quarter end'] < '7/22/2018') & (finalframe['Quarter end'] >= '4/25/2018'),
            (finalframe['Quarter end'] < '4/25/2018') & (finalframe['Quarter end'] >= '1/22/2018'),
            (finalframe['Quarter end'] < '1/22/2018') & (finalframe['Quarter end'] >= '10/22/2017'),
            (finalframe['Quarter end'] < '10/22/2017') & (finalframe['Quarter end'] >= '7/22/2017'),
            (finalframe['Quarter end'] < '7/22/2017') & (finalframe['Quarter end'] >= '4/25/2017'),
            (finalframe['Quarter end'] < '4/25/2017') & (finalframe['Quarter end'] >= '1/22/2017'),
            (finalframe['Quarter end'] < '1/22/2017') & (finalframe['Quarter end'] >= '10/22/2016'),
            (finalframe['Quarter end'] < '10/22/2016') & (finalframe['Quarter end'] >= '7/22/2016'),
            (finalframe['Quarter end'] < '7/22/2016') & (finalframe['Quarter end'] >= '4/25/2016'),
            (finalframe['Quarter end'] < '4/25/2016') & (finalframe['Quarter end'] >= '1/22/2016'),
            (finalframe['Quarter end'] < '1/22/2016') & (finalframe['Quarter end'] >= '10/22/2015'),
            (finalframe['Quarter end'] < '10/22/2015') & (finalframe['Quarter end'] >= '7/22/2015'),
            (finalframe['Quarter end'] < '7/22/2015') & (finalframe['Quarter end'] >= '4/25/2015'),
            (finalframe['Quarter end'] < '4/25/2015') & (finalframe['Quarter end'] >= '1/22/2015'),
            (finalframe['Quarter end'] < '1/22/2015') & (finalframe['Quarter end'] >= '10/22/2014'),
            (finalframe['Quarter end'] < '10/22/2014') & (finalframe['Quarter end'] >= '7/22/2014'),
            (finalframe['Quarter end'] < '7/22/2014') & (finalframe['Quarter end'] >= '4/25/2014'),
            (finalframe['Quarter end'] < '4/25/2014') & (finalframe['Quarter end'] >= '1/22/2014'),
            (finalframe['Quarter end'] < '1/22/2014') & (finalframe['Quarter end'] >= '10/22/2013'),
            (finalframe['Quarter end'] < '10/22/2013') & (finalframe['Quarter end'] >= '7/22/2013'),
            (finalframe['Quarter end'] < '7/22/2013') & (finalframe['Quarter end'] >= '4/25/2013'),
            (finalframe['Quarter end'] < '4/25/2013') & (finalframe['Quarter end'] >= '1/22/2013'),
            (finalframe['Quarter end'] < '1/22/2013') & (finalframe['Quarter end'] >= '10/22/2012'),
            (finalframe['Quarter end'] < '10/22/2012') & (finalframe['Quarter end'] >= '7/22/2012'),
            (finalframe['Quarter end'] < '7/22/2012') & (finalframe['Quarter end'] >= '4/25/2012'),
            (finalframe['Quarter end'] < '4/25/2012') & (finalframe['Quarter end'] >= '1/22/2012'),
            (finalframe['Quarter end'] < '1/22/2012') & (finalframe['Quarter end'] >= '10/22/2011'),
            (finalframe['Quarter end'] < '10/22/2011') & (finalframe['Quarter end'] >= '7/22/2011'),
            (finalframe['Quarter end'] < '7/22/2011') & (finalframe['Quarter end'] >= '4/25/2011'),
            (finalframe['Quarter end'] < '4/25/2011') & (finalframe['Quarter end'] >= '1/22/2011'),
            (finalframe['Quarter end'] < '1/22/2011') & (finalframe['Quarter end'] >= '10/22/2010'),
            (finalframe['Quarter end'] < '10/22/2010') & (finalframe['Quarter end'] >= '7/22/2010'),
            (finalframe['Quarter end'] < '7/22/2010') & (finalframe['Quarter end'] >= '4/25/2010'),
            (finalframe['Quarter end'] < '4/25/2010') & (finalframe['Quarter end'] >= '1/22/2010'),
            (finalframe['Quarter end'] < '1/22/2010') & (finalframe['Quarter end'] >= '10/22/2009'),
            (finalframe['Quarter end'] < '10/22/2009') & (finalframe['Quarter end'] >= '7/22/2009'),
            (finalframe['Quarter end'] < '7/22/2009') & (finalframe['Quarter end'] >= '4/25/2009'),
            (finalframe['Quarter end'] < '4/25/2009') & (finalframe['Quarter end'] >= '1/22/2009'),
            (finalframe['Quarter end'] < '1/22/2009') & (finalframe['Quarter end'] >= '10/22/2008'),
            (finalframe['Quarter end'] < '10/22/2008') & (finalframe['Quarter end'] >= '7/22/2008'),
            (finalframe['Quarter end'] < '7/22/2008') & (finalframe['Quarter end'] >= '4/25/2008'),
            (finalframe['Quarter end'] < '4/25/2008') & (finalframe['Quarter end'] >= '1/22/2008'),
            (finalframe['Quarter end'] < '1/22/2008') & (finalframe['Quarter end'] >= '10/22/2007'),
            (finalframe['Quarter end'] < '10/22/2007') & (finalframe['Quarter end'] >= '7/22/2007'),
            (finalframe['Quarter end'] < '7/22/2007') & (finalframe['Quarter end'] >= '4/25/2007'),
            (finalframe['Quarter end'] < '4/25/2007') & (finalframe['Quarter end'] >= '1/22/2007'),
            (finalframe['Quarter end'] < '1/22/2007') & (finalframe['Quarter end'] >= '10/22/2006'),
            (finalframe['Quarter end'] < '10/22/2006') & (finalframe['Quarter end'] >= '7/22/2006'),
            (finalframe['Quarter end'] < '7/22/2006') & (finalframe['Quarter end'] >= '4/25/2006'),
            (finalframe['Quarter end'] < '4/25/2006') & (finalframe['Quarter end'] >= '1/22/2006'),
            (finalframe['Quarter end'] < '1/22/2006') & (finalframe['Quarter end'] >= '10/22/2005'),
            (finalframe['Quarter end'] < '10/22/2005') & (finalframe['Quarter end'] >= '7/22/2005'),
            (finalframe['Quarter end'] < '7/22/2005') & (finalframe['Quarter end'] >= '4/25/2005'),
            (finalframe['Quarter end'] < '4/25/2005') & (finalframe['Quarter end'] >= '1/22/2005'),
            (finalframe['Quarter end'] < '1/22/2005') & (finalframe['Quarter end'] >= '10/22/2004'),
            (finalframe['Quarter end'] < '10/22/2004') & (finalframe['Quarter end'] >= '7/22/2004'),
            (finalframe['Quarter end'] < '7/22/2004') & (finalframe['Quarter end'] >= '4/25/2004'),
            (finalframe['Quarter end'] < '4/25/2004') & (finalframe['Quarter end'] >= '1/22/2004'),
            (finalframe['Quarter end'] < '1/22/2004') & (finalframe['Quarter end'] >= '10/22/2003'),
            (finalframe['Quarter end'] < '10/22/2003') & (finalframe['Quarter end'] >= '7/22/2003'),
            (finalframe['Quarter end'] < '7/22/2003') & (finalframe['Quarter end'] >= '4/25/2003'),
            (finalframe['Quarter end'] < '4/25/2003') & (finalframe['Quarter end'] >= '1/22/2003'),
            (finalframe['Quarter end'] < '1/22/2003') & (finalframe['Quarter end'] >= '10/22/2002'),
            (finalframe['Quarter end'] < '10/22/2002') & (finalframe['Quarter end'] >= '7/22/2002'),
            (finalframe['Quarter end'] < '7/22/2002') & (finalframe['Quarter end'] >= '4/25/2002'),
            (finalframe['Quarter end'] < '4/25/2002') & (finalframe['Quarter end'] >= '1/22/2002'),
            (finalframe['Quarter end'] < '1/22/2002') & (finalframe['Quarter end'] >= '10/22/2001'),
            (finalframe['Quarter end'] < '10/22/2001') & (finalframe['Quarter end'] >= '7/22/2001'),
            (finalframe['Quarter end'] < '7/22/2001') & (finalframe['Quarter end'] >= '4/25/2001'),
            (finalframe['Quarter end'] < '4/25/2001') & (finalframe['Quarter end'] >= '1/22/2001'),
            (finalframe['Quarter end'] < '1/22/2001') & (finalframe['Quarter end'] >= '10/22/2000'),
            (finalframe['Quarter end'] < '10/22/2000') & (finalframe['Quarter end'] >= '7/22/2000'),
            (finalframe['Quarter end'] < '7/22/2000') & (finalframe['Quarter end'] >= '4/25/2000'),
            (finalframe['Quarter end'] < '4/25/2000') & (finalframe['Quarter end'] >= '1/22/2000'),
            (finalframe['Quarter end'] < '1/22/2000') & (finalframe['Quarter end'] >= '10/22/1999'),
            (finalframe['Quarter end'] < '10/22/1999') & (finalframe['Quarter end'] >= '7/22/1999'),
            (finalframe['Quarter end'] < '7/22/1999') & (finalframe['Quarter end'] >= '4/25/1999'),
            (finalframe['Quarter end'] < '4/25/1999') & (finalframe['Quarter end'] >= '1/22/1999'),
            (finalframe['Quarter end'] < '1/22/1999') & (finalframe['Quarter end'] >= '10/22/1998'),
            (finalframe['Quarter end'] < '10/22/1998') & (finalframe['Quarter end'] >= '7/22/1998'),
            (finalframe['Quarter end'] < '7/22/1998') & (finalframe['Quarter end'] >= '4/25/1998'),
            (finalframe['Quarter end'] < '4/25/1998') & (finalframe['Quarter end'] >= '1/22/1998'),
            (finalframe['Quarter end'] < '1/22/1998') & (finalframe['Quarter end'] >= '10/22/1997'),
            (finalframe['Quarter end'] < '10/22/1997') & (finalframe['Quarter end'] >= '7/22/1997'),
            (finalframe['Quarter end'] < '7/22/1997') & (finalframe['Quarter end'] >= '4/25/1997'),
            (finalframe['Quarter end'] < '4/25/1997') & (finalframe['Quarter end'] >= '1/22/1997'),
            (finalframe['Quarter end'] < '1/22/1997') & (finalframe['Quarter end'] >= '10/22/1996'),
            (finalframe['Quarter end'] < '10/22/1996') & (finalframe['Quarter end'] >= '7/22/1996'),
            (finalframe['Quarter end'] < '7/22/1996') & (finalframe['Quarter end'] >= '4/25/1996'),
            (finalframe['Quarter end'] < '4/25/1996') & (finalframe['Quarter end'] >= '1/22/1996'),
            (finalframe['Quarter end'] < '1/22/1996') & (finalframe['Quarter end'] >= '10/22/1995'),
            (finalframe['Quarter end'] < '10/22/1995') & (finalframe['Quarter end'] >= '7/22/1995'),
            (finalframe['Quarter end'] < '7/22/1995') & (finalframe['Quarter end'] >= '4/25/1995'),
            (finalframe['Quarter end'] < '4/25/1995') & (finalframe['Quarter end'] >= '1/22/1995'),
            (finalframe['Quarter end'] < '1/22/1995') & (finalframe['Quarter end'] >= '10/22/1994'),
            (finalframe['Quarter end'] < '10/22/1994') & (finalframe['Quarter end'] >= '7/22/1994'),
            (finalframe['Quarter end'] < '7/22/1994') & (finalframe['Quarter end'] >= '4/25/1994'),
            (finalframe['Quarter end'] < '4/25/1994') & (finalframe['Quarter end'] >= '1/22/1994'),
            (finalframe['Quarter end'] < '1/22/1994') & (finalframe['Quarter end'] >= '10/22/1993'),
            (finalframe['Quarter end'] < '10/22/1993') & (finalframe['Quarter end'] >= '7/22/1993'),
            (finalframe['Quarter end'] < '7/22/1993') & (finalframe['Quarter end'] >= '4/25/1993')]

quarters_2 = ['2019Q4', '2019Q3', '2019Q2', '2019Q1', '2018Q4', '2018Q3', '2018Q2', '2018Q1',
            '2017Q4', '2017Q3', '2017Q2', '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1',
            '2015Q4', '2015Q3', '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1',
            '2013Q4', '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1',
            '2011Q4', '2011Q3', '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2', '2010Q1',
            '2009Q4', '2009Q3', '2009Q2', '2009Q1', '2008Q4', '2008Q3', '2008Q2', '2008Q1',
            '2007Q4', '2007Q3', '2007Q2', '2007Q1', '2006Q4', '2006Q3', '2006Q2', '2006Q1',
            '2005Q4', '2005Q3', '2005Q2', '2005Q1', '2004Q4', '2004Q3', '2004Q2', '2004Q1',
            '2003Q4', '2003Q3', '2003Q2', '2003Q1', '2002Q4', '2002Q3', '2002Q2', '2002Q1',
            '2001Q4', '2001Q3', '2001Q2', '2001Q1', '2000Q4', '2000Q3', '2000Q2', '2000Q1',
            '1999Q4', '1999Q3', '1999Q2', '1999Q1', '1998Q4', '1998Q3', '1998Q2', '1998Q1',
            '1997Q4', '1997Q3', '1997Q2', '1997Q1', '1996Q4', '1996Q3', '1996Q2', '1996Q1',
            '1995Q4', '1995Q3', '1995Q2', '1995Q1', '1994Q4', '1994Q3', '1994Q2', '1994Q1',
            '1993Q4', '1993Q3', '1993Q2']

finalframe['Quarter'] = np.select(periods_2, quarters_2)
finalframe.head()

In [ ]:
df2 = finalframe[finalframe.duplicated(['Symbol', 'Quarter'], keep = False)]
df2['Symbol'].unique()

## Export to .csv for edits in Excel

In [ ]:
finalframe.to_csv(interim_data, index = False)

## Import Edited File

In [2]:
path = "data\\interim\\interim_quarterly_roe_edited.csv"
df_roe = pd.read_csv(path)
df_roe.index = df_roe['Quarter end']
df_roe = df_roe.drop('Quarter end', axis=1)
df_roe.index.names=['qend']
df_roe.index = pd.to_datetime(df_roe.index)

In [12]:
df_roe

,ROE,symbol,quarter,new_key
qend,,,,
2019-09-30,0.1099,AAN,2019Q3,AAN_2019Q3
2019-06-30,0.1136,AAN,2019Q2,AAN_2019Q2
2019-03-31,0.1130,AAN,2019Q1,AAN_2019Q1
2018-12-31,0.1117,AAN,2018Q4,AAN_2018Q4
2018-09-30,0.1785,AAN,2018Q3,AAN_2018Q3
...,...,...,...,...
2014-03-30,0.5672,ZTS,2014Q1,ZTS_2014Q1
2013-12-31,NaN,ZTS,2013Q4,ZTS_2013Q4
2013-09-29,NaN,ZTS,2013Q3,ZTS_2013Q3


In [4]:
print('old shape {}'.format(df_roe.shape))
df_roe = df_roe.drop_duplicates()
print('new shape: {}'.format(df_roe.shape))

old shape (62251, 3)
new shape: (62251, 3)


In [5]:
df_roe.columns = ['ROE', 'symbol', 'quarter']

## Change Datatypes

In [6]:
df_roe['symbol'] = df_roe['symbol'].astype(str)
df_roe['quarter'] = df_roe['quarter'].astype(str)
df_roe['ROE'] = df_roe['ROE'].astype(float)

## Link Quarter to End Dates

In [7]:
df_roe['new_key'] = df_roe['symbol'] + '_'+ df_roe['quarter']

In [35]:
dates_table = pd.read_csv('data\\external\\quarter_symbols.csv')
dates_table['std_date']  = pd.to_datetime(dates_table['std_date'])
dates_table['index'] = dates_table['Symbol'] + '_' + dates_table['Quarter']

In [24]:
dates_table

,Quarter end,Symbol,Quarter,std_date,quarter,quarter date,index
0,12/31/2018,AAN,2018Q4,2018-12-31,2007Q1,3/31/2007,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,2007Q2,6/30/2007,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,2007Q3,9/30/2007,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,2007Q4,12/31/2007,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,2008Q1,3/31/2008,AAN_2017Q4
...,...,...,...,...,...,...,...
31977,3/31/2008,ZMH,2008Q1,2008-03-31,NaN,NaN,ZMH_2008Q1
31978,12/31/2007,ZMH,2007Q4,2007-12-31,NaN,NaN,ZMH_2007Q4
31979,9/30/2007,ZMH,2007Q3,2007-09-30,NaN,NaN,ZMH_2007Q3
31980,6/30/2007,ZMH,2007Q2,2007-06-30,NaN,NaN,ZMH_2007Q2


In [25]:
df_merge = pd.merge(dates_table, df_roe, left_on='index', right_on='new_key', how='inner')

In [26]:
df_merge

,Quarter end,Symbol,Quarter,std_date,quarter_x,quarter date,index,ROE,symbol,quarter_y,new_key
0,12/31/2018,AAN,2018Q4,2018-12-31,2007Q1,3/31/2007,AAN_2018Q4,0.1117,AAN,2018Q4,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,2007Q2,6/30/2007,AAN_2018Q3,0.1785,AAN,2018Q3,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,2007Q3,9/30/2007,AAN_2018Q2,0.1727,AAN,2018Q2,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,2007Q4,12/31/2007,AAN_2018Q1,0.1766,AAN,2018Q1,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,2008Q1,3/31/2008,AAN_2017Q4,0.1844,AAN,2017Q4,AAN_2017Q4
...,...,...,...,...,...,...,...,...,...,...,...
31976,3/31/2008,ZMH,2008Q1,2008-03-31,NaN,NaN,ZMH_2008Q1,0.1437,ZMH,2008Q1,ZMH_2008Q1
31977,12/31/2007,ZMH,2007Q4,2007-12-31,NaN,NaN,ZMH_2007Q4,0.1466,ZMH,2007Q4,ZMH_2007Q4
31978,9/30/2007,ZMH,2007Q3,2007-09-30,NaN,NaN,ZMH_2007Q3,0.1466,ZMH,2007Q3,ZMH_2007Q3
31979,6/30/2007,ZMH,2007Q2,2007-06-30,NaN,NaN,ZMH_2007Q2,0.1775,ZMH,2007Q2,ZMH_2007Q2


## Filter Out Symbols that only traded for the entirety of 2007-2018

In [27]:
df_07_18 = df_roe[df_roe['quarter'].str.contains('2007') | df_roe['quarter'].str.contains('2008')| df_roe['quarter'].str.contains('2009')|
           df_roe['quarter'].str.contains('2010') | df_roe['quarter'].str.contains('2011')| df_roe['quarter'].str.contains('2012')|
           df_roe['quarter'].str.contains('2013') | df_roe['quarter'].str.contains('2014')| df_roe['quarter'].str.contains('2015')|
           df_roe['quarter'].str.contains('2016') | df_roe['quarter'].str.contains('2017')| df_roe['quarter'].str.contains('2018')]

In [28]:
symbol_list = df_merge['symbol'].unique()

In [29]:
symbol_list

array(['AAN', 'AAPL', 'AAP', 'AA', 'ABBV', 'ABC', 'ABI', 'ABK', 'ABMD',
       'ABT', 'ACAS', 'ACC', 'ACE', 'ACIW', 'ACM', 'ACN', 'ACS', 'ACV',
       'ACXM', 'ADBE', 'ADCT', 'ADI', 'ADM', 'ADP', 'ADSK', 'ADS', 'ADT',
       'AEE', 'AEO', 'AEP', 'AES', 'AET', 'AFG', 'AFL', 'AGCO', 'AGN',
       'AHL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'AKRX', 'ALB', 'ALEX',
       'ALGN', 'ALK', 'ALLE', 'ALL', 'ALTR', 'ALXN', 'AMAT', 'AMCX',
       'AMD', 'AME', 'AMGN', 'AMG', 'AMP', 'AMR', 'AMSC', 'AMT', 'AMZN',
       'ANDW', 'ANF', 'ANR', 'ANSS', 'AN', 'AOC', 'AOS', 'APA', 'APC',
       'APD', 'APH', 'APOL', 'APU', 'ARE', 'ARG', 'ARRS', 'ARW', 'ASH',
       'ATI', 'ATVI', 'AVB', 'AVGO', 'AVP', 'AVY', 'AWK', 'AW', 'AXP',
       'AYE', 'AYI', 'AZO', 'A', 'BAC', 'BAX', 'BA', 'BBBY', 'BBT', 'BBY',
       'BCR', 'BC', 'BDK', 'BDX', 'BEN', 'BF.B', 'BHF', 'BHI', 'BIG',
       'BIIB', 'BJS', 'BKE', 'BK', 'BLK', 'BLL', 'BMC', 'BMS', 'BMY',
       'BNI', 'BRCM', 'BRK.A', 'BRLI', 'BRL', 'BSC', 'BSX', 'BTU', 

## ROE Table

In [30]:
df_reduced = df_merge[['std_date', 'symbol', 'ROE']]

In [31]:
df_reduced.head()

,std_date,symbol,ROE
0,2018-12-31,AAN,0.1117
1,2018-09-30,AAN,0.1785
2,2018-06-30,AAN,0.1727
3,2018-03-31,AAN,0.1766
4,2017-12-31,AAN,0.1844


In [32]:
df_merge

,Quarter end,Symbol,Quarter,std_date,quarter_x,quarter date,index,ROE,symbol,quarter_y,new_key
0,12/31/2018,AAN,2018Q4,2018-12-31,2007Q1,3/31/2007,AAN_2018Q4,0.1117,AAN,2018Q4,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,2007Q2,6/30/2007,AAN_2018Q3,0.1785,AAN,2018Q3,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,2007Q3,9/30/2007,AAN_2018Q2,0.1727,AAN,2018Q2,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,2007Q4,12/31/2007,AAN_2018Q1,0.1766,AAN,2018Q1,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,2008Q1,3/31/2008,AAN_2017Q4,0.1844,AAN,2017Q4,AAN_2017Q4
...,...,...,...,...,...,...,...,...,...,...,...
31976,3/31/2008,ZMH,2008Q1,2008-03-31,NaN,NaN,ZMH_2008Q1,0.1437,ZMH,2008Q1,ZMH_2008Q1
31977,12/31/2007,ZMH,2007Q4,2007-12-31,NaN,NaN,ZMH_2007Q4,0.1466,ZMH,2007Q4,ZMH_2007Q4
31978,9/30/2007,ZMH,2007Q3,2007-09-30,NaN,NaN,ZMH_2007Q3,0.1466,ZMH,2007Q3,ZMH_2007Q3
31979,6/30/2007,ZMH,2007Q2,2007-06-30,NaN,NaN,ZMH_2007Q2,0.1775,ZMH,2007Q2,ZMH_2007Q2


In [33]:
df_07_18

,ROE,symbol,quarter,new_key
qend,,,,
2018-12-31,0.1117,AAN,2018Q4,AAN_2018Q4
2018-09-30,0.1785,AAN,2018Q3,AAN_2018Q3
2018-06-30,0.1727,AAN,2018Q2,AAN_2018Q2
2018-03-31,0.1766,AAN,2018Q1,AAN_2018Q1
2017-12-31,0.1844,AAN,2017Q4,AAN_2017Q4
...,...,...,...,...
2014-03-30,0.5672,ZTS,2014Q1,ZTS_2014Q1
2013-12-31,NaN,ZTS,2013Q4,ZTS_2013Q4
2013-09-29,NaN,ZTS,2013Q3,ZTS_2013Q3


In [34]:
df_reduced = df_merge[['std_date', 'symbol', 'ROE']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['ROE']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

ValueError: Plan shapes are not aligned

In [ ]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape